# Preparation of frames with labels

What we do here is:
* for every annotation event get [START,STOP] interval
* merge intervals so as we have only one unique [START,STOP] interval - this means that same interval can have multiple event ids (although it's a rare situation)
* for every [START,STOP] interval calculate TIME = (START + STOP)/2
* for every TIME get all events that intersect with this point in time (contain it in their range)
* as a result for every TIME we should have a list of all events

In [1]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb
%run /home/dbaciur/NTU/NTU/notes/CommonUtils.ipynb

Constants loaded


In [5]:
import torch
import torchvision as tv

import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import intervals as I
import moviepy
from pathlib import Path
from moviepy.editor import VideoFileClip

### Load visual events

In [4]:
visual_df = pd.read_csv(C.VISUAL_EVENTS_PATH, converters=C.A_CONVERTERS)
visual_df.head()

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time
0,1,1,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,2739.504,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,132.351,2871.855,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,1502.1030
1,39,39,"S, cHarts in slides",STATE,Providing clear explanation; Arousing interest,0.0,0,3216.957,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,3433.767,6650.724,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_CHARTS_S,PH1011-PHYSICS_20150922,5042.2455
2,40,40,"P, cHarts in slides",POINT,Providing clear explanation; Arousing interest,0.0,0,0.000,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,3435.269,3435.269,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_CHARTS_P,PH1011-PHYSICS_20150922,3435.2690
3,42,42,Writing on slides,STATE,Providing clear explanation; Arousing interest,0.0,0,3145.241,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,3504.879,6650.120,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_WRITING_SLIDES,PH1011-PHYSICS_20150922,5077.4995
4,43,43,"P, cHarts in slides",POINT,Providing clear explanation; Arousing interest,0.0,0,0.000,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,3703.298,3703.298,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,B_CHARTS_P,PH1011-PHYSICS_20150922,3703.2980


### What is a frame

Frame will contain fields:
- video
- timestamp of a frame
- original event id (can be null if the frame was not generated based on any event)
- original behavior (can be null)
- state event collisions (array, can be empty)
- behaviors covered (original behavior + state event collisions' behaviors)

# Prepare frames based on point events

### Prepare frames

In [118]:
def prepare_frames_for_point_events():
    df_data = []

    for video in tqdm(visual_df[C.A_VIDEO].unique()):
        video_df = visual_df[visual_df[C.A_VIDEO] == video]
        video_point_df = video_df[video_df[C.A_BEH_ID].isin(C.POINT_BEHS)]
        video_state_df = video_df[video_df[C.A_BEH_ID].isin(C.STATE_BEHS)]

        for _, event in video_point_df.iterrows():

            event_id = event[C.A_EVENT_ID] 
            beh_id = event[C.A_BEH_ID]
            ra_id = event[C.A_RA_ID]
            timestamp = event[C.A_START]

            collision_state_events = []
            behaviors_covered = set()
            behaviors_covered.add(beh_id)

            for _, collision_event in video_state_df.iterrows():

                collision_event_start = collision_event[C.A_START]
                collision_event_stop = collision_event[C.A_STOP]

                is_collision = timestamp >= collision_event_start and timestamp <= collision_event_stop
                if not is_collision:
                    continue

                collision_state_events.append(collision_event[C.A_EVENT_ID])
                behaviors_covered.add(collision_event[C.A_BEH_ID])

            frame_id = f"{video}__{timestamp}__{event_id}__{beh_id}__{ra_id}"
            df_data.append([
                frame_id,
                video,
                timestamp,
                event_id,
                beh_id,
                ra_id,
                collision_state_events,
                list(behaviors_covered)
            ])

    return pd.DataFrame(
        data=df_data,
        columns=[
            C.F_ID,
            C.F_VIDEO,
            C.F_TIME,
            C.F_EVENT_ID,
            C.F_BEH_ID,
            C.F_RA_ID,
            C.F_COLL_EVENTS,
            C.F_BEHS
        ]
    )


In [119]:
frames_df = prepare_frames_for_point_events()

100%|██████████| 118/118 [02:39<00:00,  1.35s/it]


In [120]:
frames_df

,Id,Video,Time,Event id,Beh id,RA,Collision events,Behs
0,PH1011-PHYSICS_20150922__3435.269__40__B_CHART...,PH1011-PHYSICS_20150922,3435.269,40,B_CHARTS_P,RA13,"[39, 126]","[B_CHARTS_S, B_CHARTS_P]"
1,PH1011-PHYSICS_20150922__3703.298__43__B_CHART...,PH1011-PHYSICS_20150922,3703.298,43,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]"
2,PH1011-PHYSICS_20150922__4040.803__44__B_CHART...,PH1011-PHYSICS_20150922,4040.803,44,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]"
3,PH1011-PHYSICS_20150922__4284.434__48__B_CHART...,PH1011-PHYSICS_20150922,4284.434,48,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]"
4,PH1011-PHYSICS_20150922__4957.804__62__B_CHART...,PH1011-PHYSICS_20150922,4957.804,62,B_CHARTS_P,RA13,"[39, 42, 126]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P]"
...,...,...,...,...,...,...,...,...
5047,18S1-MH1802-LEC_20180829__2239.511__40693__B_C...,18S1-MH1802-LEC_20180829,2239.511,40693,B_CHARTS_P,RA9,"[40344, 40373, 40378, 40391, 40424, 40686]","[B_CHARTS_S, B_IMAGES_S, B_STANDS, B_WRITING_S..."
5048,18S1-MH1802-LEC_20180829__2266.546__40696__B_C...,18S1-MH1802-LEC_20180829,2266.546,40696,B_CHARTS_P,RA9,"[40344, 40373, 40378, 40391, 40424]","[B_CHARTS_S, B_IMAGES_S, B_STANDS, B_WRITING_S..."
5049,18S1-MH1802-LEC_20180829__2891.292__40765__B_C...,18S1-MH1802-LEC_20180829,2891.292,40765,B_CHARTS_P,RA9,"[40373, 40378, 40399, 40424, 40764]","[B_WRITING_SLIDES, B_CHARTS_S, B_CHARTS_P, B_S..."
5050,18S1-MH1802-LEC_20180829__4460.945__40892__B_C...,18S1-MH1802-LEC_20180829,4460.945,40892,B_CHARTS_P,RA9,"[40364, 40373, 40378, 40424, 40891]","[B_CHARTS_S, B_IMAGES_S, B_STANDS, B_WRITING_S..."


In [121]:
frames_df.to_csv(C.Frames.FRAMES_DF_DRAFT_PATH, index=False)

### Reload frames

In [5]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_DRAFT_PATH, converters=C.F_CONVERTERS)
len(frames_df)

5052

### Generate screenshots

In [ ]:
Path(C.Frames.FRAMES_SCREENSHOTS_PATH).mkdir(exist_ok=True)

In [13]:
already_generated = []
for file in os.listdir(C.Frames.FRAMES_SCREENSHOTS_PATH):
    frame_id = file[:-3]
    already_generated.append(frame_id)
    
print(already_generated[0])
print(len(already_generated))

15S1-PH1105-LEC_20150828__1988.148__2210__B_IMAGES_P__RA3
5052


In [15]:
still_to_generate = []
for frame_id in frames_df[C.F_ID].unique():
    if frame_id not in already_generated:
        still_to_generate.append(frame_id)
        
still_to_generate and print(still_to_generate)  
len(still_to_generate)

0

In [20]:
for video_name, video_df in tqdm(frames_df.groupby(C.F_VIDEO)):
    
    for i, event in video_df.iterrows():
        time = event[C.F_TIME]
        frame_name = event[C.F_ID]
        
        if frame_name not in still_to_generate:
            continue
        
        frame = tv.io.read_video(f"{C.ORIGINAL_VIDEOS_DIR}/{video_name}.mp4", start_pts=time, end_pts=time, pts_unit='sec')[0]
        torch.save(frame, f'{C.Frames.FRAMES_SCREENSHOTS_PATH}/{frame_name}.pt')

### Check behaviors distribution

In [21]:
frames_df = pd.read_csv(C.Frames.FRAMES_DF_DRAFT_PATH, converters=C.F_CONVERTERS)

behs_count = {}
for _, frame in frames_df.iterrows():
    
    beh = frame[C.F_BEH_ID]
    if beh not in behs_count:
        behs_count[beh] = 0
        
    behs_count[beh] += 1

behs_count

{'B_CHARTS_P': 2248,
 'B_IMAGES_P': 1746,
 'B_WEBSITE_P': 162,
 'B_FILMS_P': 309,
 'B_SESSIONS_P': 587}

## Remove point duplicates

In [6]:
point_df = pd.read_csv(C.Frames.POINT_DUPLICATES, converters=C.F_CONVERTERS)
len(point_df)

4920

In [8]:
frames_without_collision = []
frames_with_collision = []

for video in tqdm(point_df[C.F_VIDEO].unique()):
    
    video_point_df = point_df[point_df[C.F_VIDEO] == video]
    occupied_timestamps = []
    
    for _, frame in video_point_df.iterrows():
        
        time = frame[C.F_TIME]
        
        has_collision = False
        for t in occupied_timestamps:
            if abs(t - time) < 1:
                has_collision = True
                break
                
        if not has_collision:
            frames_without_collision.append(frame[C.F_ID])
            occupied_timestamps.append(time)
        else:
            frames_with_collision.append(frame[C.F_ID])

In [9]:
print(len(frames_without_collision))
print(len(frames_with_collision))

3854
1066


In [10]:
less_point_df = point_df[point_df[C.F_ID].isin(frames_without_collision)]
len(less_point_df)

3854

In [12]:
less_point_df.to_csv(C.Frames.POINT, index=False)